In [1]:
#default_exp api.scrum
#export
from ascrum.core.config import config
from ascrum.core.store import Store
from ascrum.core.project import Project
from ascrum.api.auth import token_required
from ascrum.server import api, start_server
from pathlib import Path
from contextlib import contextmanager
from http import HTTPStatus
from flask_restplus import Resource, Api, fields, reqparse

ascrum_ns = api.namespace('ascrum', description='Almost Scrum API')


ModuleNotFoundError: No module named 'ascrum.config'

In [2]:
#export
@api.errorhandler(ValueError)
def handle_value_error(error):
    '''Return a custom message and 400 status code'''
    return {'message': error.args[0]}, HTTPStatus.BAD_REQUEST

@api.errorhandler(PermissionError)
def handle_value_error(error):
    '''Return a custom message and 400 status code'''
    return {'message': error.args[0]}, HTTPStatus.UNAUTHORIZED


def _get_project(name, user):
    if name in config.projects:
        p = Project(Path(config.projects[name]))
        authorized_users = p.list_users()
        if user not in authorized_users:
            raise PermissionError(
                f'You have not rights to access the project {name}. '+
                f'Ask one of the existing users to grant you access, e.g. {"".join(authorized_users)}',
                authorized_users)
        return p
    else:
        return None


SyntaxError: f-string: expecting '}' (<ipython-input-2-3aea78e5a8ca>, line 20)

In [3]:
#export

project_model = api.model('Project', {
    'name': fields.String(required=True),
    'description': fields.String
})

@api.route('/projects')
class ProjectList(Resource):
    @ascrum_ns.doc('get list of projects')
    @token_required
    def get(self, user):
        return config.projects.keys()


In [4]:
#export

@api.route('/projects/<string:project_name>', endpoint='project')
@api.doc(params={
    'project_name': 'Id of a project',
    'log': 'The log where the stories are placed, sandbox, backlog, sprint-1, sprint-2...'
})
class ProjectApi(Resource):
    @ascrum_ns.doc('get project info')
    @token_required
    def get(self, project_name, user):
        p = _get_project(project_name, user)
        return {
            'name': p.name,
            'stores': p.list_stores(),
            'users': p.list_users()
        }

In [5]:
#export

story_model = api.model('Project', {
    'name': fields.String(required=True),
    'description': fields.String,
    'points': fields.Integer,
    'state': fields.String,
    'tasks': fields.List(fields.String)
})

def _is_valid_store(store):
    return store in ['sandbox', 'backlog'] or log.startswith('sprint-')
    
def _invalid_store(store):
    return f'Invalid store {store}. Valid stores are backlog, sandbox and sprint-[n]', 400
    
@api.route('/projects/<string:project_name>/<string:store>', endpoint='stories')
@api.doc(params={
    'project_name': 'Id of a project',
    'store': 'The store where the stories are placed, sandbox, backlog, sprint-1, sprint-2...'
})
class StoryList(Resource):
    @ascrum_ns.doc('get stories')
    @token_required
    def get(self, project_name: str, store_name: str, user):
        p = _get_project(project_name, user)
        if not _is_valid_store(store_name): return _invalid_store(store_name)
        return p.get_store(store).keys()

    @ascrum_ns.doc('create a new story')
    @token_required
    def post(self, project_name: str, store_name: str, user):
        p = _get_project(project_name, user)
        s = p.get_store(store_name)
        if s is None:
            return f'store {store_name} does not exist', 401
                
        move_from = self._get_move_from()
        if move_from:
            self._move_story(s, move_from)
        else:
            self._create_story(p, s, args)
            
    def _create_story(self, p, s, args):
        id = p.new_story_id()
        title = args.get('title', '')
        name = f'{id}#{title}'
        s[name] = api.payload
        return name, 201
    
    def _move_story(self, s, move_from, user):
            frm,name = move_from.split('/')
            s_from = p.get_store(frm)
            if not s_from:
                return f'store {frm} does not exist', 401
            if s_from.path == s.path:
                return f'source {frm} and target {s.name} must be different', 401
            s[name] = s_from[name]
            del s_from[name]
        
    def _get_move_from(self, user):
        parser = reqparse.RequestParser()
        parser.add_argument('move_from', type=str, help='Original story location')
        args = parser.parse_args()
        return args['move_from']

        

In [6]:

@api.route('/projects/<string:project_name>/<string:store>/<string:story_name>', endpoint='story')
@api.doc(params={
    'project_name': 'Id of a project',
    'log': 'The log where the stories are placed, sandbox, backlog, sprint-1, sprint-2...',
    'story_name': 'Story id'
})
class Story(Resource):
    @ascrum_ns.doc('get a story')
    @token_required
    def get(self, project_name: str, store: str, story_name: str, user):
        return scrum.get_story(project_name, store, story_name)


In [7]:
#hide
start_server()

 * Serving Flask app "Almost Scrum" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:4444/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2020 13:04:14] "GET /api/v1/projects/nbs HTTP/1.1" 401 -
127.0.0.1 - - [24/Oct/2020 13:04:14] "GET /api/v1/projects/nbs HTTP/1.1" 401 -
